In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [5]:
config = AutoConfig.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model.save_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
tokenizer.save_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

('finiteautomata/bertweet-base-sentiment-analysis/tokenizer_config.json',
 'finiteautomata/bertweet-base-sentiment-analysis/special_tokens_map.json',
 'finiteautomata/bertweet-base-sentiment-analysis/vocab.txt',
 'finiteautomata/bertweet-base-sentiment-analysis/bpe.codes',
 'finiteautomata/bertweet-base-sentiment-analysis/added_tokens.json')

## Import all libs

In [ ]:
import numpy as np
from scipy.special import softmax
import pandas as pd
import csv
import re
import nltk
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')

## Load emoji data for preprocessing

In [6]:
# Laden Sie den Datensatz mit den Unicode-Namen der Emojis
emoji_data = {}
with open('../../data/emoji_dataset.csv', mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        emoji_data[row[2]] = row[3]

In [7]:
def ersetze_emojis(text):
    for emoji, name in emoji_data.items():
        text = text.replace(emoji, name)
    return text

In [8]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    text = ' '.join(lemmatized_words)
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

## Test model on validation data

In [11]:
# Laden Sie Ihren CSV-Datensatz
csv_file = "../../data/val_data.csv"  # Ersetzen Sie durch den Pfad zu Ihrer CSV-Datei
df = pd.read_csv(csv_file)

# Leere Listen zum Speichern der vorhergesagten Sentiments und der tatsächlichen Labels
predicted_sentiments = []
actual_labels = []

# Durchlaufen Sie jeden Eintrag im DataFrame und wenden Sie das Modell an
for index, row in df.iterrows():
    text = row['text']
    #text = preprocess(text)  # Stellen Sie sicher, dass Sie Ihre Preprocessing-Funktion hier verwenden
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    softmax_scores = softmax(scores)

    # Extrahieren Sie das vorhergesagte Sentiment
    predicted_label_id = np.argmax(softmax_scores)
    #predicted_sentiment = model.config.id2label[predicted_label_id]

    # Speichern Sie das vorhergesagte Sentiment und das tatsächliche Label
    predicted_sentiments.append(predicted_label_id)
    actual_labels.append(row['label'])

# Fügen Sie die vorhergesagten Sentiments und die tatsächlichen Labels als neue Spalten zum DataFrame hinzu
df['predicted_sentiment'] = predicted_sentiments

# Vergleichen Sie die vorhergesagten Sentiments mit den tatsächlichen Labels
correct_predictions = (df['predicted_sentiment'] == df['label']).sum()
total_predictions = len(df)

# Berechnen Sie die Genauigkeit (Accuracy) der Vorhersagen
accuracy = correct_predictions / total_predictions

# Drucken Sie die Genauigkeit
print(f"Genauigkeit: {accuracy:.2%}")

Genauigkeit: 83.60%


In [12]:
df.head()

,text,label,predicted_sentiment
0,Dark Souls 3 April Launch Date Confirmed With ...,1,2
1,"""National hot dog day, national tequila day, t...",2,2
2,When girls become bandwagon fans of the Packer...,0,0
3,@user I may or may not have searched it up on ...,1,1
4,Here's your starting TUESDAY MORNING Line up a...,1,1


## Print <Confusion matrix

In [14]:
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Ihre Verwirrungsmatrix
conf_matrix = confusion_matrix(df['label'], df['predicted_sentiment'])

# Berechnen Sie die Precision und den Recall für jede Klasse
precision = precision_score(df['label'], df['predicted_sentiment'], average=None)
recall = recall_score(df['label'], df['predicted_sentiment'], average=None)

# Berechnen Sie den gewichteten Durchschnitt von Precision und Recall (Macro-Durchschnitt)
macro_precision = precision_score(df['label'], df['predicted_sentiment'], average='macro')
macro_recall = recall_score(df['label'], df['predicted_sentiment'], average='macro')

# Erstellen Sie ein DataFrame für die Verwirrungsmatrix
confusion_df = pd.DataFrame(conf_matrix, columns=["Predicted Class 0", "Predicted Class 1", "Predicted Class 2"], index=["True Class 0", "True Class 1", "True Class 2"])

# Drucken Sie die Verwirrungsmatrix
print("Verwirrungsmatrix:")
print(confusion_df)

# Erstellen Sie ein DataFrame für Precision und Recall
precision_recall_df = pd.DataFrame({"Precision": precision, "Recall": recall}, index=["Class 0", "Class 1", "Class 2"])

# Drucken Sie die Precision und den Recall für jede Klasse
print("\nPrecision und Recall pro Klasse:")
print(precision_recall_df)

# Drucken Sie den gewichteten Durchschnitt von Precision und Recall (Macro-Durchschnitt)
print(f"\nMacro Precision: {macro_precision:.2f}")
print(f"Macro Recall: {macro_recall:.2f}")


Verwirrungsmatrix:
              Predicted Class 0  Predicted Class 1  Predicted Class 2
True Class 0                263                 43                  6
True Class 1                 67                691                111
True Class 2                  4                 97                718

Precision und Recall pro Klasse:
         Precision    Recall
Class 0   0.787425  0.842949
Class 1   0.831528  0.795167
Class 2   0.859880  0.876679

Macro Precision: 0.83
Macro Recall: 0.84


In [16]:
precision_recall_df

,Precision,Recall
Class 0,0.787425,0.842949
Class 1,0.831528,0.795167
Class 2,0.859880,0.876679


In [15]:
confusion_df

,Predicted Class 0,Predicted Class 1,Predicted Class 2
True Class 0,263,43,6
True Class 1,67,691,111
True Class 2,4,97,718
